In [1]:
import pandas as pd
from datetime import timedelta

# Load the data
sentinel_data_path = 'sentinel_data_with_cloud_and_solar_angles2.csv'
ground_data_path = 'ground_merged_df_raw.csv'

# Read the CSV files
sentinel_df = pd.read_csv(sentinel_data_path, parse_dates=['timestamp'])
ground_df = pd.read_csv(ground_data_path, parse_dates=['Timestamp (UTC+12:00)'])
ground_df.rename(columns={'Timestamp (UTC+12:00)': 'timestamp'}, inplace=True)

# Convert columns to appropriate data types
numeric_cols = [
    'Value (m^3/s)_Discharge', 'Value (°C)_TempBuoy', 'Value (°C)_TempPlatform',
    'Value (NTU)_TurbidityBuoy', 'Value (NTU)_TurbidityPlatform',
    'Value (m)_LakeHeight', 'Value (%)_Storage', 'Value (Mm^3)_SnowVolume',
    'WDir(Deg)_EnvData', 'WSpd(m/s)_EnvData', 'GustDir(Deg)_EnvData', 
    'GustSpd(m/s)_EnvData', 'WindRun(Km)_EnvData', 'Rain(mm)_EnvData', 
    'Tdry(C)_EnvData', 'TWet(C)_EnvData', 'RH(%)_EnvData', 'Tmax(C)_EnvData', 
    'Tmin(C)_EnvData', 'Pmsl(hPa)_EnvData', 'Pstn(hPa)_EnvData'
]

for col in numeric_cols:
    ground_df[col] = pd.to_numeric(ground_df[col], errors='coerce')

# Separate ground data based on sampling intervals
three_hour_avg_cols = ['Value (m^3/s)_Discharge']
fifteen_min_avg_cols = [
    'Value (°C)_TempBuoy', 'Value (°C)_TempPlatform',
    'Value (NTU)_TurbidityBuoy', 'Value (NTU)_TurbidityPlatform'
]
daily_avg_cols = [
    'Value (m)_LakeHeight', 'Value (%)_Storage', 'Value (Mm^3)_SnowVolume',
    'WDir(Deg)_EnvData', 'WSpd(m/s)_EnvData', 'GustDir(Deg)_EnvData', 
    'GustSpd(m/s)_EnvData', 'WindRun(Km)_EnvData', 'Rain(mm)_EnvData', 
    'Tdry(C)_EnvData', 'TWet(C)_EnvData', 'RH(%)_EnvData', 'Tmax(C)_EnvData', 
    'Tmin(C)_EnvData', 'Pmsl(hPa)_EnvData', 'Pstn(hPa)_EnvData'
]

# Function to find the closest ground measurement to a given Sentinel-2 timestamp
def find_closest_measurements(sentinel_time, ground_df, cols, time_window):
    # Define the time window
    start_time = sentinel_time - time_window
    end_time = sentinel_time + time_window
    # Filter ground data within the time window
    filtered_ground_df = ground_df[(ground_df['timestamp'] >= start_time) & (ground_df['timestamp'] <= end_time)]
    # Find the closest timestamp for each column
    closest_rows = {}
    for col in cols:
        if col in filtered_ground_df:
            closest_time = filtered_ground_df['timestamp'].iloc[(filtered_ground_df['timestamp'] - sentinel_time).abs().argsort()[:1]]
            if not closest_time.empty:
                closest_rows[col] = filtered_ground_df[filtered_ground_df['timestamp'] == closest_time.iloc[0]][col].values[0]
    return closest_rows

# Define the time window for finer intervals (e.g., 1 hour)
time_window = timedelta(hours=1)

# Prepare a list to store merged data
merged_data = []

# Process each Sentinel-2 timestamp
for index, row in sentinel_df.iterrows():
    sentinel_time = row['timestamp']
    merged_row = row.to_dict()
    
    # Find closest measurements for three hour average and fifteen minute interval columns
    closest_three_hour = find_closest_measurements(sentinel_time, ground_df, three_hour_avg_cols, time_window)
    closest_fifteen_min = find_closest_measurements(sentinel_time, ground_df, fifteen_min_avg_cols, time_window)
    
    # Update the merged row with these measurements
    merged_row.update(closest_three_hour)
    merged_row.update(closest_fifteen_min)
    
    merged_data.append(merged_row)

# Convert the merged data to a DataFrame
merged_df = pd.DataFrame(merged_data)

# Create date columns for merging daily averages
ground_df['date'] = ground_df['timestamp'].dt.date
sentinel_df['date'] = sentinel_df['timestamp'].dt.date
merged_df['date'] = pd.to_datetime(merged_df['timestamp']).dt.date  # Ensure 'date' column is present in merged_df

# Calculate daily averages for relevant ground data
daily_avg_df = ground_df.groupby('date')[daily_avg_cols].mean().reset_index()

# Merge the daily averages with the Sentinel-2 data
merged_df = pd.merge(merged_df, daily_avg_df, on='date', how='left')

# Drop the 'date' column as it was only for merging purposes
merged_df.drop(columns=['date'], inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_sentinel_ground_data2.csv', index=False)

print("Merging completed. The merged dataset is saved as 'merged_sentinel_ground_data2.csv'")

/var/folders/q3/x0b5k_7d55ld8ywy_cs0vhy80000gn/T/ipykernel_92958/2616420234.py:10: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  ground_df = pd.read_csv(ground_data_path, parse_dates=['Timestamp (UTC+12:00)'])


KeyError: 'Value (m^3/s)_Discharge'

In [6]:
merged_df.describe()

,timestamp,B11_0,B11_1,B11_2,B11_3,B11_4,B11_5,B11_6,B11_7,B11_8,...,GustSpd(m/s)_EnvData,WindRun(Km)_EnvData,Rain(mm)_EnvData,Tdry(C)_EnvData,TWet(C)_EnvData,RH(%)_EnvData,Tmax(C)_EnvData,Tmin(C)_EnvData,Pmsl(hPa)_EnvData,Pstn(hPa)_EnvData
count,780,780.000000,777.000000,774.000000,773.000000,773.000000,773.000000,772.000000,768.000000,765.000000,...,763.000000,772.000000,771.000000,774.000000,774.000000,774.000000,773.000000,772.000000,774.000000,774.000000
mean,2021-09-20 02:27:50.998718208,1728.205128,1760.799228,1767.217054,1802.890039,1802.890039,1743.258732,1805.049223,1744.167969,1779.111111,...,9.473132,229.176166,2.107393,9.695995,7.203359,69.522351,16.053816,6.228497,1013.825711,966.443282
min,2018-12-16 10:28:16,0.000000,9.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.600000,94.000000,0.000000,-1.600000,-2.600000,17.000000,2.900000,-3.800000,972.700000,926.200000
25%,2020-05-16 16:35:51.750000128,92.750000,130.000000,131.000000,151.000000,151.000000,96.000000,151.750000,88.000000,144.000000,...,6.200000,174.000000,0.000000,6.600000,4.200000,56.825000,11.700000,3.100000,1006.800000,959.725000
50%,2021-09-20 10:33:23,898.500000,956.000000,962.000000,1159.000000,1159.000000,936.000000,1159.000000,967.000000,1024.000000,...,8.200000,209.500000,0.000000,9.350000,7.200000,73.350000,16.100000,6.200000,1014.300000,967.150000
75%,2023-01-27 22:35:50,3124.750000,3177.000000,3183.000000,3212.000000,3212.000000,3182.000000,3212.500000,3168.500000,3210.000000,...,10.800000,261.000000,1.200000,12.700000,10.000000,84.475000,19.700000,9.200000,1021.300000,973.475000
max,2024-06-10 10:38:28,8173.000000,7900.000000,7900.000000,7931.000000,7931.000000,8173.000000,7931.000000,7925.000000,7896.000000,...,34.500000,1064.000000,73.600000,26.600000,18.600000,94.300000,31.300000,20.300000,1038.700000,989.800000
std,NaN,1872.983495,1853.928838,1854.645417,1829.853415,1829.853415,1874.723168,1830.054083,1877.102455,1843.736479,...,4.807619,91.238801,5.640621,4.732831,3.979721,17.506554,5.444568,4.097209,10.591700,9.915802


In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Columns: 112 entries, timestamp to Pstn(hPa)_EnvData
dtypes: datetime64[ns](1), float64(111)
memory usage: 682.6 KB


In [9]:
print(merged_df.isna().sum())

timestamp            0
B11_0                0
B11_1                3
B11_2                6
B11_3                7
                    ..
RH(%)_EnvData        6
Tmax(C)_EnvData      7
Tmin(C)_EnvData      8
Pmsl(hPa)_EnvData    6
Pstn(hPa)_EnvData    6
Length: 112, dtype: int64
